# SPCS Networking Connectivity Test: Google Drive

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# Google Drive Connectivity Test Configuration

# Test URLs
GOOGLE_API_ADMIN_URL = "https://admin.googleapis.com"
GOOGLE_API_OAUTH_URL = "https://oauth2.googleapis.com"
GOOGLE_API_ID_URL = "https://google.com"

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

print(f"Configuration:")
print(f"Google API Admin: {GOOGLE_API_ADMIN_URL}")
print(f"Google API OAuth: {GOOGLE_API_OAUTH_URL}")
print(f"Google API ID: {GOOGLE_API_ID_URL}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("GOOGLE DRIVE CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['google_api_admin'] = test_connectivity(GOOGLE_API_ADMIN_URL, "Google API Admin")
results['google_api_oauth'] = test_connectivity(GOOGLE_API_OAUTH_URL, "Google API OAuth")
results['google_api_id'] = test_connectivity(GOOGLE_API_ID_URL, "Google API ID Service")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required Google Drive endpoints.")
    print("You can proceed with Google Drive integration.")
else:
    print(f"\n⚠️  Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

If Connectivity testing fails, you can use the Cells below to prepare and implement an EAI suitable for Google Drive access.

In [ ]:
-- Create Network Rules for Google Drive connectivity
-- Run this cell if connectivity tests failed

-- Network rule for Google Drive authentication services
CREATE OR REPLACE NETWORK RULE gdrive_connectivity_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = (
    'admin.googleapis.com',
    'oauth2.googleapis.com',
    'www.googleapis.com',
    'google.com'
  );

SHOW NETWORK RULES LIKE 'gdrive_%';

In [ ]:
use role accountadmin;
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION gdrive_eai
  ALLOWED_NETWORK_RULES = (
    gdrive_connectivity_rule
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Google Drive connectivity';

  SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'gdrive_eai';

In [ ]:
ALTER NOTEBOOK EAI_GDRIVE
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('GDRIVE_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.

*NOTE: Remember to restart your compute session*

In [ ]:
-- Check if network rules exist
SHOW NETWORK RULES LIKE 'gdrive_%';

In [ ]:
-- Check if EAI exists
SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'gdrive_eai';